In [ ]:
#***********************************************************************************
#GRAPHICAL INTERFACE FOR EMOTIONAL DETECTION IN TWITCH CHATS IN THE GAMING ENVIRONMENT
#***********************************************************************************

import tkinter as tk
from tkinter import ttk
from tkinter import Label
from tkinter import filedialog  # Importing filedialog from tkinter
from tkinter import PhotoImage
from PIL import Image, ImageTk # Importing Pillow to resize
import pandas as pd
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.text import Tokenizer
import pickle  # Importing the pickle library
from keras.utils import pad_sequences
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import load_model
import nltk
from nltk.tokenize import word_tokenize
import string
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.utils import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from collections import Counter
from tkinter import messagebox

# Globals variables for storing the DataFrame
comments_df = None
results_df = None
comments_df_sentiment = None
results_df_sentiment = None

#***********************************************************************************
#--------------------------------------FUNCTIONS AT THE TOP OF THE INTERFACE--------
#***********************************************************************************

#***********************************************************************************
# Function that executes the algorithm to predict the emotions in a sentence that we write in the interface
#***********************************************************************************
def ejecutar_algoritmo(input_texts):
    # Load the pre-trained model and the tokeniser
    model_path = r".\modelo_entrenado_twitch\model_robertuito_emociones"
    tokenizer_path = r".\modelo_entrenado_twitch\tok_robertuito_emociones"

    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    # Tokenisation of data
    inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt")
    
    # Pasar los tokens a través del modelo
    with torch.no_grad():
        outputs = model(**inputs)

    # Processing predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_labels = torch.argmax(probabilities, dim=-1)
    
    # Converting predictions to a Python list
    predicted_labels_list = predicted_labels.cpu().numpy().tolist()
    
    # Map predicted labels to their corresponding text labels
    id2label = {0: "Disappointment/Sadness", 1: "Approval/Empathy", 2: "Anger/Rage",
                    3: "Interest/Hype", 4: "Disapproval", 5: "Neutral"}
    
    #Label Aprobación is Approval/Empathy/Confidence
    #Label Desaprobación is Disapproval
    #Label Decepción/Tristeza is Disappointment/Sadnes
    #Label Enfado/Ira is Anger
    #Label Interés/Aceptación/Hype is Interest/Acceptance/Hype
    #Label Indeterminado is Neutral

    predicted_labels_text = [id2label[label_id] for label_id in predicted_labels_list]
    
    
    print("The predicted class for the phrase is:", predicted_labels_text)
    resultado = predicted_labels_text
    return resultado

#*************************************************************************************************************
# Function that executes the algorithm to predict the emotions in a sentence that we write in the interface
#*************************************************************************************************************
def ejecutar_algoritmo_sentiment(input_texts_sentiment):
    # Load the pre-trained model and the tokeniser
    model_path = r".\modelo_entrenado_twitch\model_robertuito_polaridad"
    tokenizer_path = r".\modelo_entrenado_twitch\tok_robertuito_polaridad"

    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    
    # Tokenisation of data
    inputs = tokenizer(input_texts_sentiment, padding=True, truncation=True, return_tensors="pt")
    
    # Passing tokens through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Processing predictions
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_labels = torch.argmax(probabilities, dim=-1)
    
    # Converting predictions to a Python list
    predicted_labels_list = predicted_labels.cpu().numpy().tolist()
    
    # Map predicted labels to their corresponding text labels
    id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}
    #Label Positivo is Positive
    #Label Negativo is Negative
    #Label Indeterminado is Neutral
    
    predicted_labels_text = [id2label[label_id] for label_id in predicted_labels_list]

    print("The predicted class for the phrase is:", predicted_labels_text)
    resultado_sentiment = predicted_labels_text
    return resultado_sentiment

#***********************************************************************************
# Function to push a button and run the emotion detection algorithm
#***********************************************************************************
def boton_algoritmo():
    global resultado
    input_texts = cuadro_texto_frame1.get()
    print("Comments:", input_texts)
    ejecutar_algoritmo(input_texts)
    resultado = ejecutar_algoritmo(input_texts)
    print(resultado)
    resultado_strvar.set(resultado)
    cuadro_prediccion_resultado = ttk.Entry(frame1,textvariable=resultado_strvar,width=30,state="disabled")  
    cuadro_prediccion_resultado.pack()
    cuadro_prediccion_resultado.place(x=550, y=60)

#***********************************************************************************
# Function to push a button and run the polarity detection algorithm
#***********************************************************************************
def boton_algoritmo_sentiment():
    global resultado_sentiment
    input_texts_sentiment = cuadro_texto_frame2.get()
    print("Comentarios:", input_texts_sentiment)
    ejecutar_algoritmo_sentiment(input_texts_sentiment)
    resultado_sentiment = ejecutar_algoritmo_sentiment(input_texts_sentiment)
    print(resultado_sentiment)
    resultado_sentiment_strvar.set(resultado_sentiment)
    cuadro_prediccion_resultado_sentiment = ttk.Entry(frame2,textvariable=resultado_sentiment_strvar,width=30,state="disabled")  
    cuadro_prediccion_resultado_sentiment.pack()
    cuadro_prediccion_resultado_sentiment.place(x=550, y=60)


#***********************************************************************************
#--------------------------------------FUNCTIONS AT THE BOTTON OF THE INTERFACE--------
#***********************************************************************************

#***********************************************************************************
# Function to show information regarding the file to predict
#***********************************************************************************
def mostrar_informacion():
    messagebox.showinfo("IMPORTANT", "The file must contain in the first column the following comments.")

#***********************************************************************************
# File upload function to predict emotional response(csv/xlss)
#***********************************************************************************
def cargar_archivo():
    global comments_df 
    global ruta_archivo
    maxlen=100000000000
    # Open a dialogue box to select the file
    ruta_archivo = filedialog.askopenfilename(filetypes=[("Excel workbook", "*.xlsx"),("Excel workbook 97- 2004", "*.xls"),("Archivo CSV", "*.csv")])
    
    # Read the selected file 
    if ruta_archivo:
        try:
            # Determine file type based on file extension
            if ruta_archivo.endswith('.xls') or ruta_archivo.endswith('.xlsx'):
                # Upload Excel file
                comments_df = pd.read_excel(ruta_archivo, usecols=[0], names=["Comment"], engine='openpyxl')
                #to display the temporary message
                mostrar_mensaje()

            elif ruta_archivo.endswith('.csv'):
                # Load the CSV file
                comments_df = pd.read_csv(ruta_archivo, delimiter=';', usecols=[0], header=None, skiprows=1, names=["Comentario"], encoding="latin1")
                #to display the temporary message 
                mostrar_mensaje()
            else:
                # Display an error message if the file is not supported
                print("Error: The selected file is not an Excel workbook or a CSV file.")
                return

            comments_df.drop(comments_df.head(0).index,inplace = True)
            print(comments_df.head(2))  # Displays the first rows of the DataFrame
            #enable the button to run the algorithm
            boton_ejecut_csv = ttk.Button(frame1, text="Run Algorithm with Csv/Excel", command=ejecutar_excel,padding=(10, 10),style='ColorBoton.TButton',state="normal")
            #boton_ejecut_csv.pack(pady=0) #indicates the space between this button and the next button 
            boton_ejecut_csv.place(x=550, y=150)
        
        except Exception as e:
            # Handle any errors that may occur when uploading the file
            print("Error uploading file:", e)
        
    return comments_df 

#***********************************************************************************
# File upload function to predict the polarity response (csv/xlss)
#***********************************************************************************
def cargar_archivo_sentiment():
    global comments_df_sentiment
    global ruta_archivo_sentiment
    maxlen=100000000000
    # Open a dialogue box to select the file
    ruta_archivo_sentiment = filedialog.askopenfilename(filetypes=[("Excel workbook", "*.xlsx"),("Excel workbook 97- 2004", "*.xls"),("Archivo CSV", "*.csv")])
    
    # Read the selected file  
    if ruta_archivo_sentiment:
        try:
            # Determine file type based on file extension
            if ruta_archivo_sentiment.endswith('.xls') or ruta_archivo_sentiment.endswith('.xlsx'):
                # Upload Excel file
                comments_df_sentiment = pd.read_excel(ruta_archivo_sentiment, usecols=[0], names=["Comment"], engine='openpyxl')
                #to display the temporary message
                mostrar_mensaje()

            elif ruta_archivo_sentiment.endswith('.csv'):
                # Upload CSV file
                comments_df_sentiment = pd.read_csv(ruta_archivo_sentiment, delimiter=';', usecols=[0], header=None, skiprows=1, names=["Comentario"], encoding="latin1")
                #to display the temporary message 
                mostrar_mensaje()
            else:
                # Display an error message if the file is not supported
                print("Error: The selected file is not an Excel workbook or a CSV file.")
                return

            comments_df_sentiment.drop(comments_df_sentiment.head(0).index,inplace = True)
            print(comments_df_sentiment.head(2))  # Muestra las primeras filas del DataFrame
            
            #enable the button to run the algorithm
            boton_ejecut_csv_sentiment = ttk.Button(frame2, text="Run Algorithm with Csv/Excel", command=ejecutar_excel_sentiment,padding=(10, 10),style='ColorBoton.TButton',state="normal")
            #boton_ejecut_csv.pack(pady=0) #indicates the space between this button and the next button 
            boton_ejecut_csv_sentiment.place(x=550, y=150)
        
        except Exception as e:
            # Handle any errors that may occur when uploading the file
            print("Error uploading file:", e)
        
    return comments_df_sentiment

#*************************************************************************************************************
# Function that executes the algorithm to predict the emotions in a upload file (csv/xlsx)
#*************************************************************************************************************
def ejecutar_excel():
    global comments_df 
    global ruta_archivo
    global results_df
    
    if comments_df is not None:

       # Load the pre-trained model and the tokeniser
        model_path = r".\modelo_entrenado_twitch\model_robertuito_emociones"
        tokenizer_path = r".\modelo_entrenado_twitch\tok_robertuito_emociones"
        
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

        # Get the comments as a list
        input_texts = comments_df["Comentario"].tolist()
        
        # Data tokenisation
        inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt")
        
        # Pass tokens through the model
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Procesar las predicciones
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_labels = torch.argmax(probabilities, dim=-1)
        
        # Converting predictions to a Python list
        predicted_labels_list = predicted_labels.cpu().numpy().tolist()
        
        # Map predicted labels to their corresponding text labels
        id2label = {0: "Disappointment/Sadness", 1: "Approval/Empathy", 2: "Anger/Rage",
                    3: "Interest/Hype", 4: "Disapproval", 5: "Neutral"}        
        #Label Aprobación is Approval/Empathy/Confidence
        #Label Desaprobación is Disapproval
        #Label Decepción/Tristeza is Disappointment/Sadnes
        #Label Enfado/Ira is Anger
        #Label Interés/Aceptación/Hype is Interest/Acceptance/Hype
        #Label Indeterminado is Neutral
        
        predicted_labels_text = [id2label[label_id] for label_id in predicted_labels_list]
        
        # Show predictions
        print("Predictions:")
        for text, label in zip(input_texts, predicted_labels_text):
            print(f"Comments: {text.strip()}, Prediction: {label}")

        # Create a DataFrame of pandas with the original comments, predictions and tags (if available in the CSV file)
        results_df = pd.DataFrame({'Comments': input_texts, 'Prediction': predicted_labels_text})
        print(results_df.head(3))

        # Display the graphs
        mostrar_grafica1(results_df)
        mostrar_grafica2(results_df)

        #enable the button to download the file with the predictions
         boton_descargar_csv_habilitado = ttk.Button(frame1, text="Download Excel/CSV", command=descargar_excel,padding=(43, 10),style='ColorBoton.TButton',state="normal")
        boton_descargar_csv_habilitado.pack(pady=0) #indicates the space between this button and the next one
        boton_descargar_csv_habilitado.place(x=1100, y=150)
        
        return results_df

#*************************************************************************************************************
# Function that executes the algorithm to predict the polarity in a upload file (csv/xlsx)
#*************************************************************************************************************
def ejecutar_excel_sentiment():
    global comments_df_sentiment
    global ruta_archivo_sentiment
    global results_df_sentiment
    
    if comments_df_sentiment is not None:

       # Load the pre-trained model and the tokeniser
        model_path = r".\modelo_entrenado_twitch\model_robertuito_polaridad"
        tokenizer_path = r".\modelo_entrenado_twitch\tok_robertuito_polaridad"
        
        model = AutoModelForSequenceClassification.from_pretrained(model_path)
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

        # Get the comments as a list
        input_texts_sentiment = comments_df["Comentario"].tolist()
        
        # Tokenisation of data
        inputs = tokenizer(input_texts_sentiment, padding=True, truncation=True, return_tensors="pt")
        
        # Passing tokens through the model
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Processing predictions
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_labels = torch.argmax(probabilities, dim=-1)
        
        # Converting predictions to a Python list
        predicted_labels_list = predicted_labels.cpu().numpy().tolist()
        
        # Map predicted labels to their corresponding text labels
        id2label = {0: "Negative", 1: "Neutral", 2: "Positive"}
        #Label Positivo is Positive
        #Label Negativo is Negative
        #Label Indeterminado is Neutral

        predicted_labels_text = [id2label[label_id] for label_id in predicted_labels_list]
        
        # Show predictions
        print("Predictions:")
        for text, label in zip(input_texts_sentiment, predicted_labels_text):
            print(f"Comments: {text.strip()}, Prediction: {label}")

        # Create a DataFrame of pandas with the original comments, predictions and tags (if available in the CSV file)
        results_df_sentiment = pd.DataFrame({'Comments': input_texts_sentiment, 'Prediction': predicted_labels_text})
        print(results_df_sentiment.head(3))

        # Display the graphs
        mostrar_grafica3(results_df_sentiment)
        mostrar_grafica4(results_df_sentiment)

        #enable the download button 
        boton_descargar_csv_habilitado_sentiment = ttk.Button(frame2, text="Download Excel/CSV", command=descargar_excel_sentiment,padding=(43, 10),style='ColorBoton.TButton',state="normal")
        boton_descargar_csv_habilitado_sentiment.place(x=1100, y=150)
        
        return results_df

#*************************************************************************************************************
# Function to download the excel with the emotional predictions (csv/xlsx)
#*************************************************************************************************************
def descargar_excel():
    global results_df
    
    # Open a dialogue box to select location and file name
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel workbook", "*.xlsx"), ("CSV File", "*.csv")])
    
    # Check whether the user cancelled the transaction
    if file_path:
        try:
        
            # Save the DataFrame in a file according to the selected extension
            if  file_path.endswith('.xlsx'): #== '.xlsx':
                results_df.to_excel(file_path, index=False)
                
            elif file_path.endswith('.csv'): #== '.csv':
                results_df.to_csv(file_path, index=False)
            else:
                # Display an error message if the file is not supported
                print("Error: The selected file is not an Excel workbook or a CSV file.")
                return
        except Exception as e:
            # Handle any errors that may occur when uploading the file
            print("Error uploading file:", e)

            
#*************************************************************************************************************
# Function to download the excel with the polarity predictions (csv/xlsx)
#*************************************************************************************************************
def descargar_excel_sentiment():
    global results_df_sentiment
    
    # Open a dialogue box to select location and file name
    file_path_sentiment = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel workbook", "*.xlsx"), ("CSV File", "*.csv")])
    
    # Check whether the user cancelled the transaction
    if file_path:
        try:
        
            # Save the DataFrame in a file according to the selected extension
            if  file_path_sentiment.endswith('.xlsx'): #== '.xlsx':
                results_df_sentiment.to_excel(file_path, index=False)
                
            elif file_path_sentiment.endswith('.csv'): #== '.csv':
                results_df_sentiment.to_csv(file_path, index=False)
            else:
                # Display an error message if the file is not supported
                print("Error: The selected file is not an Excel workbook or a CSV file.")
                return
        except Exception as e:
            # Handle any errors that may occur when uploading the file
            print("Error uploading file:", e)


def mostrar_mensaje():
    label.config(text="File uploaded successfully")
 
    frame1.after(2000, borrar_mensaje)  # 5000 miliseconds = 5 seconds
    frame2.after(2000, borrar_mensaje)  # 5000 miliseconds = 5 secons

def borrar_mensaje():
    label.config(text="")

#*************************************************************************************************************
#  Function to display the graph with the percentage of the polarity of the comments in the file
#*************************************************************************************************************
def mostrar_grafica1(results_df):
    # Get count of unique values in ‘Predicted’ column
    class_counts = results_df['Prediction'].value_counts()

    # Calculate the percentage of each class
    total_samples = len(results_df)
    class_percentages = (class_counts / total_samples) * 100

    # Create the bar chart
    plt.figure(figsize=(5,4))
    class_percentages.plot(kind='bar', color='pink')
    plt.title('Distribution of Comments on Emotions (%)',fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('Emotion',fontweight='bold')
    plt.ylabel('Percentage (%)',fontweight='bold')
    plt.tight_layout()

    # Display the graph in frame1
    canvas = FigureCanvasTkAgg(plt.gcf(), master=frame1)
    canvas.draw()
    canvas.get_tk_widget().place(x=50, y=220)

            
#*************************************************************************************************************
# Function for displaying the graph of the real-time evolution of the emotions of the comments in the file
#*************************************************************************************************************
def mostrar_grafica2(results_df):
    # Define the class-to-value mapping dictionary   
    clase_valor = {
        'Interest/Hype': 6,
        'Approval/Empathy': 4,
        'Disapproval': 2,
        'Neutral': 0,
        'Disappointment/Sadness': -2,
        'Anger/Rage': -4
    }
    
    # Create a list to store the x and y coordinates of each point
    coordenadas_x = []
    coordenadas_y = []

    for _, row in results_df.iterrows():
        clase = row['Prediction']
        valor = clase_valor.get(clase, 0)
        coordenadas_y.append(valor)
        coordenadas_x.append(_)

    plt.figure(figsize=(7, 4))
    plt.plot(coordenadas_x, coordenadas_y, marker='o', color='blue', linewidth=0.5, markersize=4)

    # Configure the y-axis labels
    plt.yticks(list(clase_valor.values()), list(clase_valor.keys()))

    plt.title('Distribution of all Comments on Emotions', fontweight='bold')
    plt.xlabel('Number of comments', fontweight='bold')
    plt.ylabel('Emotion', fontweight='bold')
    plt.subplots_adjust(left=0.3)
    plt.subplots_adjust(bottom=0.25)
    plt.yticks(rotation=40)

    
    # Display the graph in frame1
    canvas = FigureCanvasTkAgg(plt.gcf(), master=frame1)
    canvas.draw()
    canvas.get_tk_widget().place(x=600, y=220)

    plt.show()

#*************************************************************************************************************
#  Function to display the graph with the percentage of the polarity of the comments in the file
#*************************************************************************************************************
def mostrar_grafica3(results_df_sentiment):
    # Get count of unique values in ‘Predicted’ column
    class_counts = results_df_sentiment['Prediction'].value_counts()

    # Calculate the percentage of each class
    total_samples = len(results_df_sentiment)
    class_percentages = (class_counts / total_samples) * 100

    # Create the bar chart
    plt.figure(figsize=(5,4))
    class_percentages.plot(kind='bar', color='pink')
    plt.title('Distribution of Comments on Polarity (%)',fontweight='bold')
    plt.xlabel('Polarity',fontweight='bold')
    plt.ylabel('Percentage (%)',fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    # Display the graph in frame2
    canvas = FigureCanvasTkAgg(plt.gcf(), master=frame2)
    canvas.draw()
    canvas.get_tk_widget().place(x=50, y=220)

            
#*************************************************************************************************************
# Function for displaying the graph of the real-time evolution of the polarity of the comments in the file
#*************************************************************************************************************
def mostrar_grafica4(results_df_sentiment):
    # Define the class-to-value mapping dictionary    
    clase_valor = {
        'Positive': 2,
        'Neutral': 0,
        'Negative': -2,
    }
    
    # Create a list to store the x and y coordinates of each point
    coordenadas_x = []
    coordenadas_y = []
    etiquetas_y = []  # List for storing the texts associated with each value on the y-axis
    
    for _, row in results_df_sentiment.iterrows():
        clase = row['Prediction']
        valor = clase_valor.get(clase, 0)
        coordenadas_y.append(valor)
        coordenadas_x.append(_)

    plt.figure(figsize=(7, 4))
    plt.plot(coordenadas_x, coordenadas_y, marker='o', color='blue', linewidth=0.5, markersize=4)

    # Configure the y-axis labels
    plt.yticks(list(clase_valor.values()), list(clase_valor.keys()))

    plt.xlabel('Number of comments', fontweight='bold')
    plt.ylabel('Polarity', fontweight='bold')
    plt.title('Distribution of all Comments on Polarity', fontweight='bold')
    plt.subplots_adjust(left=0.3)
    plt.subplots_adjust(bottom=0.25)
    plt.yticks(rotation=40)

    canvas = FigureCanvasTkAgg(plt.gcf(), master=frame2)
    canvas.draw()
    canvas.get_tk_widget().place(x=600, y=220)

    plt.show()
#*************************************************************************************************************
#---------------------------------------------------END OF FUNCTIONS-------------------------------------
#*************************************************************************************************************
#*************************************************************************************************************



#---------------------------SEPARATOR LINE ------------------------
def draw_separator(canvas):
    # Draw a continuous horizontal line in the centre of the canvas
    canvas.create_line(30, 100, 1320, 100, fill="darkblue", width=2)
    

#*************************************************************************************************************
#------------------INTERFACE----------------#
#*************************************************************************************************************
# Create an instance of the main window
ventana = tk.Tk()

# Obtain the dimensions of the screen
screen_width = ventana.winfo_screenwidth()
screen_height = ventana.winfo_screenheight()

ventana.title("Emotional Response for Twich and Video Games")
ventana.configure(background='#7c1324')

resultado_strvar = tk.StringVar()
resultado_sentiment_strvar = tk.StringVar()

# to change the window dimensions
#it has to be put in front of geometry()
ventana.resizable(width=False, height=False)

# Set window dimensions
ventana.geometry(f"{screen_width}x{screen_height}")

# Create a Style object
estilo = ttk.Style()
# Setting the background colour of the tabs
estilo.configure("Fondo.TFrame", background="lightblue")

# Create a Notebook widget (tabs)
pestanas = ttk.Notebook(ventana)

# Create the content of each tab
frame1 = ttk.Frame(pestanas)
frame2 = ttk.Frame(pestanas)

# Adding content to each tab
pestanas.add(frame1, text="Emotions")
pestanas.add(frame2, text="Polarity")

# Packing the Notebook
pestanas.pack(expand=1, fill="both")

# Setting the background colour of the tabs
estilo.configure("TNotebook.Tab", highlightbackground='ADD8E6')

# Create a custom button style
estilo = ttk.Style()
estilo.configure('ColorBoton.TButton', background='#ADD8E6')

#-----------------Create an INFORMATION BUTTON
boton_informacion = ttk.Button(ventana, text="Info", command=mostrar_informacion)
boton_informacion.pack(pady=10)
boton_informacion.place(x=240, y=185)

#------------------------Create the canvas for the DIVIDING LINE 
canvas1 = tk.Canvas(frame1, width=2200, height=800)
canvas1.pack()
canvas2 = tk.Canvas(frame2, width=2200, height=800)
canvas2.pack()
# Draw the separation line in the frame
draw_separator(canvas1)
draw_separator(canvas2)

try:
    imagen = Image.open('./Logo3VideoGames.jpeg')
    imagen = imagen.resize((180, 180), resample=Image.BICUBIC)
    foto = ImageTk.PhotoImage(imagen)
    
    # Create a label to display the image
    label_imagen1 = tk.Label(frame1, image=foto, width=180, height=180)
    label_imagen1.place(x=1325, y=600)
    label_imagen2 = tk.Label(frame2, image=foto, width=180, height=180)
    label_imagen2.place(x=1325, y=600)
except FileNotFoundError:
    print("Error: Image not found")
    exit()
    
#----------------------------------------------Top (Emotions --> frame1) -----------------------------

titulo_frame1 = ttk.Label(frame1, text="Insert a comment:")
titulo_frame1.place(x=50, y=40)
cuadro_texto_frame1 = ttk.Entry(frame1, width=30)
cuadro_texto_frame1.place(x=50, y=60)

titulo_prediccion_frame1 = ttk.Label(frame1, text="Comment prediction:")
titulo_prediccion_frame1.place(x=550, y=40)
cuadro_prediccion_frame1 = ttk.Entry(frame1, width=30, state="disabled")  
cuadro_prediccion_frame1.place(x=550, y=60)

boton_algoritmo_frame1 = ttk.Button(frame1, text="Run Algorithm", command=boton_algoritmo, padding=(43, 10))
boton_algoritmo_frame1.place(x=1100, y=40)

#----------------------------------------------BOTTOM (Emotions --> frame1) -----------------------------

botoncsv_frame1 = ttk.Button(frame1, text="Insert csv/Excel", command=cargar_archivo, padding=(40, 10), style='ColorBoton.TButton')
botoncsv_frame1.place(x=50, y=150)

# Button
boton_ejecut_csv_frame1 = ttk.Button(frame1, text="Run Algorithm with Csv/Excel", command=ejecutar_excel, padding=(10, 10), style='ColorBoton.TButton', state="disabled")
boton_ejecut_csv_frame1.place(x=550, y=150)

boton_descargar_csv_frame1 = ttk.Button(frame1, text="Download Excel/CSV", command=descargar_excel, padding=(43, 10), style='ColorBoton.TButton', state="disabled")
boton_descargar_csv_frame1.place(x=1100, y=150)

#---------------------------------TOP (Polarity --> frame2) -----------------------------

titulo_frame2 = ttk.Label(frame2, text="Insert a comment:")
titulo_frame2.place(x=50, y=40)
cuadro_texto_frame2 = ttk.Entry(frame2, width=30)
cuadro_texto_frame2.place(x=50, y=60)

titulo_prediccion_frame2 = ttk.Label(frame2, text="Comment prediction:")
titulo_prediccion_frame2.place(x=550, y=40)
cuadro_prediccion_frame2 = ttk.Entry(frame2, width=30, state="disabled")  
cuadro_prediccion_frame2.place(x=550, y=60)

boton_algoritmo_frame2 = ttk.Button(frame2, text="Run Algorithm", command=boton_algoritmo_sentiment, padding=(43, 10))
boton_algoritmo_frame2.place(x=1100, y=40)

#----------------------------------------------BOTTOM (Polarity --> frame2) -----------------------------

botoncsv_frame2 = ttk.Button(frame2, text="Insert csv/Excel", command=cargar_archivo_sentiment, padding=(40, 10), style='ColorBoton.TButton')
botoncsv_frame2.place(x=50, y=150)

# Buttons
boton_ejecut_csv_frame2 = ttk.Button(frame2, text="Run Algorithm with Csv/Excel", command=ejecutar_excel_sentiment, padding=(10, 10), style='ColorBoton.TButton', state="disabled") 
boton_ejecut_csv_frame2.place(x=550, y=150)

boton_descargar_csv_frame2 = ttk.Button(frame2, text="Download Excel/CSV", command=descargar_excel_sentiment, padding=(43, 10), style='ColorBoton.TButton', state="disabled")
boton_descargar_csv_frame2.place(x=1100, y=150)

#label to display temporary messages
label = tk.Label(frame1, text="")
label= tk.Label(frame2, text="")



# Start event loop
ventana.mainloop()


                                          Comentario
0   nivel 80? lo tenia bien guardado el vicio gen...
1                                                HUH
